# Manage vectorDB collections 

Use this notebook to list / delete collections within Milvus DB and check schemas and number of rows of a collection

Example Usage: 

List
| makeresults
| fit MLTKContainer algo=llm_rag_milvus_management task=list_collections _time into app:llm_rag_milvus_management as RAG
| eval RAG_Collections=split(RAG_Collections,"|") | mvexpand RAG_Collections

Delete
| makeresults
| fit MLTKContainer algo=llm_rag_milvus_management task=delete_collection collection_name=document_collection_example _time into app:llm_rag_milvus_management as RAG

Schema
| makeresults
| fit MLTKContainer algo=llm_rag_milvus_management task=show_schema collection_name=document_collection_example _time into app:llm_rag_milvus_management as RAG

Rows
| makeresults
| fit MLTKContainer algo=llm_rag_milvus_management task=show_rows collection_name=document_collection_example _time into app:llm_rag_milvus_management as RAG


## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [15]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
import urllib
import pymilvus
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)
from pymilvus import MilvusClient
from pinecone import Pinecone, ServerlessSpec
from llama_index_alloydb_pg import AlloyDBEngine, AlloyDBVectorStore, AlloyDBDocumentStore, AlloyDBIndexStore
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [7]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("llm_rag_milvus_management")

## Stage 2 - create and initialize a model

In [10]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    model['hyperparameter'] = 42.0
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

## Stage 3 - fit the model

In [12]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [13]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

{'message': 'model trained'}


## Stage 4 - apply the model

In [14]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    # Manager task includes list_collections, delete_collection, show_schema and show_rows
    task = param['options']['params']['task']
    try:
        collection_name = param['options']['params']['collection_name']
    except:
        collection_name = 'default_collection'

    try:
        vectordb_service = param['options']['params']['vectordb_service']
    except:
        vectordb_service = 'milvus'
    if vectordb_service == 'milvus':
        uri = json.loads(os.environ['llm_config'])['vector_db']['milvus'][0]['uri']
        token = json.loads(os.environ['llm_config'])['vector_db']['milvus'][0]['token']
        host = urllib.parse.urlparse(uri).hostname
        port = urllib.parse.urlparse(uri).port
        client = MilvusClient(
            uri=uri,
            token=token
        )
        
        connections.connect("default", host=host, port=port)
        
        if task == "general":
            collection_list = client.list_collections()
            schemas = []
            rows = []
            for clt in collection_list:
                collection = Collection(clt)
                m1 = str([item.name for item in collection.schema.fields])
                schemas.append(m1)
                rows.append(collection.num_entities)
            cols = {"Collections": collection_list, "Fields": schemas, "Number_of_rows": rows}
                
        elif task == "list_collections":
            collection_list = "|".join(client.list_collections())
            cols = {"Collections": [collection_list], "Message": ["No result"], "Schema": ["No result"], "Number_of_rows": ["No result"]}
        elif task == "delete_collection":
            utility.drop_collection(collection_name)
            m = f'Deleted collection {collection_name}'
            cols = {"Collections": ["No result"], "Message": [m], "Schema": ["No result"], "Number_of_rows": ["No result"]}
        elif task == "show_schema":
            try:
                collection = Collection(collection_name)
                s = str(collection.schema.fields)
            except:
                s = ''
            cols = {"Collections": ["No result"], "Message": ["No result"], "Schema": [s], "Number_of_rows": ["No result"]}
        elif task == "show_rows":
            try:
                collection = Collection(collection_name)
                n = str(collection.num_entities)
            except:
                n = ''
            cols = {"Collections": ["No result"], "Message": ["No result"], "Schema": ["No result"], "Number_of_rows": [n]}
        else:
            cols = {"Collections": ["No result"], "Message": ["No result"], "Schema": ["No result"], "Number_of_rows": ["No result"]}
    elif vectordb_service == 'pinecone':
        api_key = json.loads(os.environ['llm_config'])['vector_db']['pinecone'][0]['api_key']
        pc = Pinecone(api_key=api_key)
            
        if task == "list_collections":
            try:
                collection_list = []
                index_list = pc.list_indexes()['indexes']
                for item in index_list:
                    collection_list.append(item['name'])
                collection_list = "|".join(collection_list)
            except:
                collection_list = ""
            cols = {"Collections": [collection_list], "Message": ["No result"], "Schema": ["No result"], "Number_of_rows": ["No result"]}
                
        else:
            cols = {"Collections": ["No result"], "Message": ["Task not supported for pinecone."], "Schema": ["No result"], "Number_of_rows": ["No result"]}

    else:
        if task == "list_collections":
             cols = {"Collections": ["No result"], "Message": ["Task not supported for alloyDB. Please specify collection names manually."], "Schema": ["No result"], "Number_of_rows": ["No result"]}   
        else:
            cols = {"Collections": ["No result"], "Message": ["Task not supported for alloyDB."], "Schema": ["No result"], "Number_of_rows": ["No result"]}

        
    result = pd.DataFrame(data=cols)
    return result

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

## Stage 5 - save the model

In [16]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [17]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [18]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

def compute(model,df,param):
    # Manager task includes list_collections, delete_collection, show_schema and show_rows
    task = param['options']['params']['task']
    try:
        collection_name = param['options']['params']['collection_name']
    except:
        collection_name = 'default_collection'

    try:
        vectordb_service = param['options']['params']['vectordb_service']
    except:
        vectordb_service = 'milvus'
    if vectordb_service == 'milvus':
        uri = json.loads(os.environ['llm_config'])['vector_db']['milvus'][0]['uri']
        token = json.loads(os.environ['llm_config'])['vector_db']['milvus'][0]['token']
        host = urllib.parse.urlparse(uri).hostname
        port = urllib.parse.urlparse(uri).port
        client = MilvusClient(
            uri=uri,
            token=token
        )
        
        connections.connect("default", host=host, port=port)
        
        if task == "general":
            collection_list = client.list_collections()
            schemas = []
            rows = []
            for clt in collection_list:
                collection = Collection(clt)
                m1 = str([item.name for item in collection.schema.fields])
                schemas.append(m1)
                rows.append(collection.num_entities)
            cols = {"Collections": collection_list, "Fields": schemas, "Number_of_rows": rows}
                
        elif task == "list_collections":
            collection_list = "|".join(client.list_collections())
            cols = {"Collections": [collection_list], "Message": ["No result"], "Schema": ["No result"], "Number_of_rows": ["No result"]}
        elif task == "delete_collection":
            utility.drop_collection(collection_name)
            m = f'Deleted collection {collection_name}'
            cols = {"Collections": ["No result"], "Message": [m], "Schema": ["No result"], "Number_of_rows": ["No result"]}
        elif task == "show_schema":
            try:
                collection = Collection(collection_name)
                s = str(collection.schema.fields)
            except:
                s = ''
            cols = {"Collections": ["No result"], "Message": ["No result"], "Schema": [s], "Number_of_rows": ["No result"]}
        elif task == "show_rows":
            try:
                collection = Collection(collection_name)
                n = str(collection.num_entities)
            except:
                n = ''
            cols = {"Collections": ["No result"], "Message": ["No result"], "Schema": ["No result"], "Number_of_rows": [n]}
        else:
            cols = {"Collections": ["No result"], "Message": ["No result"], "Schema": ["No result"], "Number_of_rows": ["No result"]}
    elif vectordb_service == 'pinecone':
        api_key = json.loads(os.environ['llm_config'])['vector_db']['pinecone'][0]['api_key']
        pc = Pinecone(api_key=api_key)
            
        if task == "list_collections":
            try:
                collection_list = []
                index_list = pc.list_indexes()['indexes']
                for item in index_list:
                    collection_list.append(item['name'])
                collection_list = "|".join(collection_list)
            except:
                collection_list = ""
            cols = {"Collections": [collection_list], "Message": ["No result"], "Schema": ["No result"], "Number_of_rows": ["No result"]}
                
        else:
            cols = {"Collections": ["No result"], "Message": ["Task not supported for pinecone."], "Schema": ["No result"], "Number_of_rows": ["No result"]}

    else:
        if task == "list_collections":
             cols = {"Collections": ["No result"], "Message": ["Task not supported for alloyDB. Please specify collection names manually."], "Schema": ["No result"], "Number_of_rows": ["No result"]}   
        else:
            cols = {"Collections": ["No result"], "Message": ["Task not supported for alloyDB."], "Schema": ["No result"], "Number_of_rows": ["No result"]}

        
    result = pd.DataFrame(data=cols)
    return result

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## Send data back to Splunk HEC
When you configured the Splunk HEC Settings in the DSDL app you can easily send back data to an index with [Splunk's HTTP Event Collector (HEC)](https://docs.splunk.com/Documentation/Splunk/latest/Data/UsetheHTTPEventCollector). Read more about data formats and options in the [documentation](https://docs.splunk.com/Documentation/Splunk/latest/Data/FormateventsforHTTPEventCollector#Event_metadata).

### Use cases
- you want to offload longer running, possibly distributed computations that need to deliver results asynchroneously back into Splunk. 
- you might not want to present results back into the search pipeline after your `| fit` or `| apply` command. 
- you can easily utilize this approach for any logging purposes or other profiling tasks in your ML code so you can actively monitor and analyze your processes.

### Example

In [18]:
from dsdlsupport import SplunkHEC as SplunkHEC
hec = SplunkHEC.SplunkHEC()

In [19]:
# example to send 10 hello world events
response = hec.send_hello_world(10)

In [20]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


In [21]:
# example to send a JSON object, e.g. to log some data
from datetime import datetime
response = hec.send({'event': {'message': 'operation done', 'log_level': 'INFO' }, 'time': datetime.now().timestamp()})

In [22]:
print("HEC endpoint %s \nreturned with status code %s \nand response message: %s" % (response.url, response.status_code, response.text))

HEC endpoint http://host.docker.internal:8088/services/collector/event 
returned with status code 200 
and response message: {"text":"Success","code":0}


## End of Stages
All subsequent cells are not tagged and can be used for further freeform code